# Delta Water Unavailability Methodology Module 4: Unavailability Analysis

In [1]:
# This script calls the outputs of the 1.Supply and 2.Demand scripts to analyze water unavailability
# for all rights and claims in the Sacramento-San Joaquin Delta Waterhsed
# Further documentation is available on the Delta Water Unavailability Methodology website:
# https://www.waterboards.ca.gov/waterrights/water_issues/programs/drought/drought_tools_methods/delta_method.html

# Import necessary packages
import pandas as pd
import numpy as np
import os
from IPython.display import display, Markdown

In [2]:
# Define common lists of subwatersheds
headwater_subwatershed = ['Sacramento Bend', 'Stony', 'Cache', 'Upper Feather', 'Yuba', 'Bear', 'Upper American', 'Putah', 
                 'Chowchilla', 'Upper San Joaquin', 'Fresno', 'Merced', 'Tuolumne', 'Stanislaus', 'Calaveras', 'Mokelumne',
                'Cosumnes']
subwatersheds = ['Sacramento Bend', 'Stony', 'Cache', 'Upper Feather', 'Yuba', 'Bear', 'Upper American', 'Putah', 
                 'Upper Sacramento Valley', 'Sacramento Valley Floor', 'Chowchilla', 'Upper San Joaquin', 'Fresno',
                 'Merced', 'Tuolumne', 'Stanislaus', 'Calaveras', 'Mokelumne', 'Cosumnes', 'San Joaquin Valley Floor']

# Define non-headwater subwatersheds
subwatersheds_to_check = ['Upper Sacramento Valley', 'Sacramento Valley Floor', 'San Joaquin Valley Floor']

# Define subtotals of multiple subwatersheds
summary_subtotals = ['Sacramento Subtotal', 'San Joaquin Subtotal', 'Delta Tributary Subtotal']

In [3]:
# Function to check if required input files are present
def check_required_outputs(folder_path, files_to_notebooks):
    # Check if the folder exists
    if not os.path.exists(folder_path):
        print(f"Required folder 'intermediate-outputs' does not exist. Please run the Supply and Demand notebooks first.")
        return False  # Indicates the required folder does not exist
    
    # If there are specific files we want to check for within the folder
    missing_instructions = []
    for file, notebook in files_to_notebooks.items():
        if not os.path.exists(os.path.join(folder_path, file)):
            missing_instructions.append(f"{file} (Please run {notebook})")

    if missing_instructions:
        missing_str = "\n".join(missing_instructions)
        print(f"The following required files are missing:\n{missing_str}")
        return False  # Indicates one or more required files do not exist
    
    # If the folder exists and all required files are present
    print("All required intermediate outputs are present. Proceeding with analysis.")
    return True  # Indicates all checks passed

In [4]:
# Check if the intermediate outputs generated by previous scripts are present
files_to_notebooks = {
    'Analysis_Dataset_Pre.csv': 'Demand.ipynb',
    'Demands_Period_POD.csv': 'Demand.ipynb',
    'Supply_Period_Total.csv': 'Supply.ipynb',
    'Instream_Flow_Period.csv': 'Supply.ipynb'
}

if not check_required_outputs('./intermediate-outputs', files_to_notebooks):
    display(Markdown("**Execution stopped. Please generate the required intermediate outputs before proceeding.**"))

All required intermediate outputs are present. Proceeding with analysis.


In [5]:
# Import pre-defined list of aggregated Riparian claims to check number of rows
prior = pd.read_csv('../user-inputs/AnalysisPreparationRiparian.csv')

# Import Analysis, POD Demands, Total Supply, and Instream Flows datasets from 1.Supply and 2.Demand scripts
analysis_dataset = pd.read_csv('./intermediate-outputs/Analysis_Dataset_Pre.csv')
demand_period_pod = pd.read_csv('./intermediate-outputs/Demands_Period_POD.csv')
supply_period_total = pd.read_csv('./intermediate-outputs/Supply_Period_Total.csv')
instream_flow_period = pd.read_csv('./intermediate-outputs/Instream_Flow_Period.csv')

## A) Headwater Analysis

In [6]:
'''
Calculate Total Demand
a) For the first 81 rows, sum POD Period Demand for the respective 
   Water Right Type, Subwatershed, and Legal_Delta combination

b) For the remaining records, sum POD Period Demand for the respective
   Application ID, Subwatershed, and Legal_Delta combination
'''

# Group by the necessary columns and sum 'DEMAND_PERIOD_POD'
grouped_demand_first = demand_period_pod.groupby(['WATER_RIGHT_TYPE_CUSTOM', 'SUBWATERSHED', 'LEGAL_DELTA'])['DEMAND_PERIOD_POD'].sum().reset_index()
grouped_demand_rest = demand_period_pod.groupby(['APPL_ID', 'SUBWATERSHED', 'LEGAL_DELTA'])['DEMAND_PERIOD_POD'].sum().reset_index()

# For the first rows of aggregated Riparian demand
analysis_dataset_first_part = pd.merge(
    analysis_dataset.iloc[:len(prior)],
    grouped_demand_first,
    on=['WATER_RIGHT_TYPE_CUSTOM', 'SUBWATERSHED', 'LEGAL_DELTA'],
    how='left'
)
analysis_dataset_first_part.rename(columns={'DEMAND_PERIOD_POD': 'DEMAND_TOTAL'}, inplace=True)
analysis_dataset_first_part['DEMAND_TOTAL'].fillna(0, inplace=True)
  
# Skip the first rows of aggregated Riparian demand, that was taken care of in a) above
analysis_dataset_remaining_part = pd.merge(
    analysis_dataset.iloc[len(prior):],
    grouped_demand_rest,
    on=['APPL_ID', 'SUBWATERSHED', 'LEGAL_DELTA'],
    how='left'
)
analysis_dataset_remaining_part.rename(columns={'DEMAND_PERIOD_POD': 'DEMAND_TOTAL'}, inplace=True)

# Concatenate the two parts
analysis_dataset = pd.concat([analysis_dataset_first_part, analysis_dataset_remaining_part], ignore_index=True)

In [7]:
# Create new Hierarchical column before calculation to match previous conventions
analysis_dataset_heir = analysis_dataset.reindex(columns = analysis_dataset.columns.tolist()
                                  + headwater_subwatershed*3 + (subwatersheds+summary_subtotals) + subwatersheds*2)

# Create columns for elements of the water unavailability analysis
# 17 headwater subwatersheds + 3 lower subwatersheds + 3 subtotals
analysis_dataset_heir.columns = [['']*9 + ['DEMAND_HEADWATER']*17 +['SUPPLY_HEADWATER_ACCT']*17 
                                 +['DEMAND_MET_HEADWATER']*17 +['SUPPLY_WATERSHED_ACCT']*23 +
                                 ['DEMAND_WATERSHED']*20 + ['DEMAND_MET_WATERSHED']*20, analysis_dataset_heir.columns]

In [8]:
'''
For each record, calculate a Headwater Demand from each of the 17 headwater subwatersheds
'''

# Calculate how much of each demand is in each headwater subwatershed
for subwatershed in headwater_subwatershed:
    condition = (analysis_dataset_heir['', 'HEADWATER']) & \
                (~analysis_dataset_heir['', 'LEGAL_DELTA']) & \
                (analysis_dataset_heir['', 'SUBWATERSHED'] == subwatershed)

    analysis_dataset_heir[('DEMAND_HEADWATER', subwatershed)] = np.where(condition, 
                                                                          analysis_dataset_heir['', 'DEMAND_TOTAL'], 
                                                                          0)

In [9]:
'''
Calculate a running Headwater Supply Account for each of the 17 headwater subwatersheds:
a) For the first record, equal to its Supply Period Total
b) For each subsequent record, max(0, previous Headwater Supply Account – previous Headwater Demand)
   In other words, subtract the previous demand from the previous supply until there is no more supply left
'''

# Set the first row to each headwater subwatershed's Supply Period Total
for subwatershed in headwater_subwatershed:
    # Get the corresponding Supply Period Total for the respective subwatershed
    initial_supply = supply_period_total.loc[supply_period_total['SUBWATERSHED'] == subwatershed, 'SUPPLY_PERIOD_TOTAL'].iloc[0]
    analysis_dataset_heir.at[0, ('SUPPLY_HEADWATER_ACCT', subwatershed)] = initial_supply
    
    # Convert relevant columns to numpy arrays for vectorized operations
    supply_headwater_acct_array = analysis_dataset_heir[('SUPPLY_HEADWATER_ACCT', subwatershed)].to_numpy()
    demand_headwater_array = analysis_dataset_heir[('DEMAND_HEADWATER', subwatershed)].to_numpy()

# Calculate subsequent account balances
    for index in range(1, len(analysis_dataset_heir)):
        supply_headwater_acct_array[index] = max(0, supply_headwater_acct_array[index-1] - demand_headwater_array[index-1])
    
    analysis_dataset_heir[('SUPPLY_HEADWATER_ACCT', subwatershed)] = supply_headwater_acct_array

In [10]:
'''
For each record, calculate a Headwater Demand Met from each of the 17 headwater subwatersheds: 
Headwater Supply Account – max(0, Headwater Supply Account – Headwater Demand Met) 
In other words, the lesser of either the headwater demand or the headwater supply account remaining
'''

# Calculate Headwater Demand Met for each record from each headwater subwatershed
for subwatershed in headwater_subwatershed:
    demand_headwater_col = ('DEMAND_HEADWATER', subwatershed) 
    supply_headwater_acct_col = ('SUPPLY_HEADWATER_ACCT', subwatershed)
    demand_met_headwater_col = ('DEMAND_MET_HEADWATER', subwatershed)

    supply_headwater_acct = analysis_dataset_heir[supply_headwater_acct_col]
    demand_headwater = analysis_dataset_heir[demand_headwater_col]
    
    analysis_dataset_heir[demand_met_headwater_col] = supply_headwater_acct - \
                                                     pd.Series([max(0, supply - demand) for supply, demand in zip(supply_headwater_acct, demand_headwater)],
                                                               index=analysis_dataset_heir.index)

## B) Headwater Analysis Outputs

In [11]:
'''
For each subwatershed, determine a Disconected value
If a headwater subwatershed's total Riparian demand exceeds its supply, it is considered temporarily disconnected
and its supplies and demands will not carry over to the watershed-scale analysis 
'''

# Create a list and dataframe for the expected Headwater Analysis Outputs
outputs = ['SUPPLY_PERIOD_TOTAL', 'RIPARIAN_DEMAND_TOTAL', 'DISCONNECTED', 'DEMAND_UNMET_RIPARIAN',
          'INSTREAM_FLOW_EXCEEDS', 'INSTREAM_FLOW_SUPPLY', 'SUPPLY_PERIOD_FINAL']
hwa_outputs = pd.DataFrame(index=outputs, columns=subwatersheds, dtype=object)

for subwatershed in subwatersheds:
    subwatershed_col = ('', 'SUBWATERSHED')
    legal_delta_col = ('', 'LEGAL_DELTA')
    demand_total_col = ('', 'DEMAND_TOTAL')
    
    # Get the corresponding Period Total Supply for the current subwatershed
    supply_period_total_value = supply_period_total[supply_period_total['SUBWATERSHED'] == subwatershed]['SUPPLY_PERIOD_TOTAL'].values[0]
    hwa_outputs.loc['SUPPLY_PERIOD_TOTAL', subwatershed] = supply_period_total_value
    
    # Sum Total Demand values for records with the same Subwatershed value and Riparian Priority Dates
    riparian_demand_sum = 0
    
    # Only look in the first rows of aggregated Riparian demand
    for x in range(len(prior)):
        if analysis_dataset_heir.loc[x, subwatershed_col]==subwatershed and analysis_dataset_heir.loc[x, legal_delta_col]==False:
            riparian_demand_sum += analysis_dataset_heir.loc[x, demand_total_col]
    hwa_outputs.loc['RIPARIAN_DEMAND_TOTAL', subwatershed] = riparian_demand_sum
    
    # Determine a Disconnected value for each subwatershed
    if subwatershed in subwatersheds_to_check:
        hwa_outputs.loc['DISCONNECTED', subwatershed] = False
    elif riparian_demand_sum > supply_period_total_value:
        hwa_outputs.loc['DISCONNECTED', subwatershed] = True
    else:
        hwa_outputs.loc['DISCONNECTED', subwatershed] = False

In [12]:
'''
For each subwatershed, sum Unmet Riparian Demands
'''

for subwatershed in subwatersheds:
    if subwatershed not in subwatersheds_to_check:
        # Sum Riparian-priority claims that are not Riparian-only, as they have access to Instream Flow Supply
        mask_subwatershed = (analysis_dataset_heir[('', 'SUBWATERSHED')] == subwatershed)
        mask_riparian = (analysis_dataset_heir[('', 'WATER_RIGHT_TYPE_CUSTOM')] == "Statement of Div and Use (Riparian or Pre-1914)") | \
                        (analysis_dataset_heir[('', 'WATER_RIGHT_TYPE_CUSTOM')] == "Statement of Div and Use (Other)")

        demand_unmet_riparian = \
            analysis_dataset_heir.loc[mask_subwatershed & mask_riparian, ('DEMAND_HEADWATER', subwatershed)].sum() - \
            analysis_dataset_heir.loc[mask_subwatershed & mask_riparian, ('DEMAND_MET_HEADWATER', subwatershed)].sum()
        
    else:
        # Unmet Riparian Demands in lower subwatersheds aren't accounted for in this step
        demand_unmet_riparian = 0
        
    # Determine Unmet Riparian Demand for each subwatershed
    hwa_outputs.loc['DEMAND_UNMET_RIPARIAN', subwatershed] = demand_unmet_riparian

In [13]:
'''
For each subwatershed, check if Instream Flow Exceeds subwatershed supply (T/F)
For each subwatershed, calculate the volume of Instream Flow Supply in excess of subwatershed supply
'''

for subwatershed in subwatersheds:
    # Get the corresponding Period Supply Total for each subwatershed
    supply_period_total_value = supply_period_total.loc[
        supply_period_total['SUBWATERSHED'] == subwatershed, 'SUPPLY_PERIOD_TOTAL'].values[0]
    
    # Get the corresponding Instream Flow Period for each subwatershed
    instream_flow_period_value = instream_flow_period.loc[
        instream_flow_period['SUBWATERSHED'] == subwatershed, 'INSTREAM_FLOW_PERIOD'].values[0]

    # Check if Supply Period Total is less than Instream Flow Period
    instream_flow_exceeds_value = supply_period_total_value < instream_flow_period_value
    
    if instream_flow_exceeds_value:
        instream_flow_supply_value = instream_flow_period_value - supply_period_total_value
    else:
        # INSTREAM_FLOW_SUPPLY is zero when INSTREAM_FLOW_EXCEEDS is False
        instream_flow_supply_value = 0
    
    hwa_outputs.loc['INSTREAM_FLOW_EXCEEDS', subwatershed] = instream_flow_exceeds_value
    hwa_outputs.loc['INSTREAM_FLOW_SUPPLY', subwatershed] = instream_flow_supply_value

In [14]:
'''
For each subwatershed, calculate a Supply Period Final
incorporating Supply Period Total, Disconnected, Unmet Riparian Demands, and Instream Flow Supply values
'''

for subwatershed in subwatersheds:
    disconnected_value = hwa_outputs.loc['DISCONNECTED', subwatershed]
    instream_flow_supply_value = hwa_outputs.at['INSTREAM_FLOW_SUPPLY', subwatershed]
    demand_unmet_riparian_value = hwa_outputs.at['DEMAND_UNMET_RIPARIAN', subwatershed]
    
    # Get the corresponding Supply Period Total for each subwatershed
    supply_period_total_value = supply_period_total.loc[
        supply_period_total['SUBWATERSHED'] == subwatershed, 'SUPPLY_PERIOD_TOTAL'].values[0]
    if disconnected_value:
        # If Disconnected, only Instream Flow Supply will contribute to watershed-scale analysis
        # But Riparians have access to Instream Flow Supply,
        # so Supply Period Final is zero if Unmet Riparian Demands exceed Instream Flow Supply
        supply_period_final_value = max(0, instream_flow_supply_value - demand_unmet_riparian_value)
    else:
        # If not Disconnected, add Supply Period Total and Instream Flow Supply
        supply_period_final_value = supply_period_total_value + instream_flow_supply_value    
    hwa_outputs.loc['SUPPLY_PERIOD_FINAL', subwatershed] = supply_period_final_value

#display(hwa_outputs)

In [15]:
# Export intermediate Headwater Calculations for watershed-scale analysis
hwa_outputs.to_csv('./intermediate-outputs/Headwater_Calculations.csv')

#### Headwater Analysis Results

In [16]:
'''
For each record, calculate a Total Headwater Demand Met:
Sum of Headwater Demand Met from all 17 headwater subwatersheds
'''

disconnected_values = {subwatershed: hwa_outputs.loc['DISCONNECTED', subwatershed] for subwatershed in headwater_subwatershed}
is_headwater_not_legal_delta = analysis_dataset_heir[('', 'HEADWATER')] & ~analysis_dataset_heir[('', 'LEGAL_DELTA')]
subwatershed_values = analysis_dataset_heir[('', 'SUBWATERSHED')].map(disconnected_values)
demand_met_headwater_sum = analysis_dataset_heir.filter(like='DEMAND_MET_HEADWATER').sum(axis=1)

# Calculate Total Headwater Demand Met for each record
analysis_dataset_heir[('HEADWATER_ANALYSIS_RESULTS', 'DEMAND_MET_HEADWATER_TOTAL')] = np.where(
    is_headwater_not_legal_delta,
    np.where(subwatershed_values, 0, demand_met_headwater_sum),
    analysis_dataset_heir[('', 'DEMAND_TOTAL')]
)

In [17]:
'''
For each record, calculate a Total Headwater Unmet Demand:
Total Demand – Total Headwater Demand Met
'''

# Calculate Total Headwater Unmet Demand for each record
analysis_dataset_heir[('HEADWATER_ANALYSIS_RESULTS', 'DEMAND_UNMET_HEADWATER_TOTAL')] = 0

analysis_dataset_heir[('HEADWATER_ANALYSIS_RESULTS', 'DEMAND_UNMET_HEADWATER_TOTAL')] = (
    analysis_dataset_heir[('', 'DEMAND_TOTAL')] - 
    analysis_dataset_heir[('HEADWATER_ANALYSIS_RESULTS', 'DEMAND_MET_HEADWATER_TOTAL')]
)

In [18]:
'''
For each record, determine a Water Unavailable in Headwater (T/F) value:
a) If the record is located in a headwater area,
   outside the Legal Delta,
   and the Headwater Supply Account for the Subwatershed where it is located is zero, 
   then equal to “TRUE”
b) If any of the above 3 conditions aren’t met, equal to “FALSE”
'''

# Determine if Water Unavaiable in Headwater for each record
analysis_dataset_heir['HEADWATER_ANALYSIS_RESULTS', 'WATER_UNAVAILABLE_HEADWATER'] = False
analysis_dataset_heir[('HEADWATER_ANALYSIS_RESULTS', 'WATER_UNAVAILABLE_HEADWATER')] = False
analysis_dataset_heir.loc[is_headwater_not_legal_delta, ('HEADWATER_ANALYSIS_RESULTS', 'WATER_UNAVAILABLE_HEADWATER')] = \
    analysis_dataset_heir.loc[is_headwater_not_legal_delta].apply(lambda row: row[('SUPPLY_HEADWATER_ACCT', row[('', 'SUBWATERSHED')])] == 0, axis=1)

## C) Watershed Analysis

In [ ]:
'''
(1) Calculate a running Watershed Supply Account for all 20 subwatersheds:
    a) For the first record, equal to its Supply Period Final
    b) For each subsequent record, max(0, previous Watershed Supply Account – previous Watershed Demand)
       In other words, subtract the previous demand from the previous supply until there is no more supply left

(2) For each record, calculate a Watershed Demand from each of the 20 subwatersheds. 
    The demands of each record are charged against the source subwatersheds that can supply them,
    with each subwatershed distributed in a unique way based on the assumed connectivity of the watershed
    Only demands that can be met in the headwater-scale analysis are included
'''

# (1) and (2) are calculated together since they depend on eachother

#Define function to handle division by zero if any subtotal is 0
def safe_divide(numerator, denominator):
    return numerator / denominator if denominator != 0 else 0

# Dictionary mapping subwatersheds to their corresponding subtotals
subtotal_mapping = {
    'Sacramento Subtotal': ['Sacramento Bend', 'Stony', 'Cache', 'Upper Feather', 'Yuba', 'Bear', 
                            'Upper American', 'Upper Sacramento Valley', 'Sacramento Valley Floor'],
    'San Joaquin Subtotal': ['Upper San Joaquin', 'Fresno', 'Chowchilla', 'Merced', 'Tuolumne',
                             'San Joaquin Valley Floor'],
    'Delta Tributary Subtotal': ['Putah', 'Stanislaus', 'Calaveras', 'Mokelumne', 'Cosumnes']
}

# Loop through each record in the Analysis dataset
for index in range(len(analysis_dataset_heir)):
    water_right_type_custom = analysis_dataset_heir.at[index, ('', 'WATER_RIGHT_TYPE_CUSTOM')]
    subwatershed_value = analysis_dataset_heir.at[index, ('', 'SUBWATERSHED')]
    watershed_value = analysis_dataset_heir.at[index, ('', 'WATERSHED')]
    legal_delta_value = analysis_dataset_heir.at[index, ('', 'LEGAL_DELTA')]
    headwater_value = analysis_dataset_heir.at[index, ('', 'HEADWATER')]
    demand_met_headwater_value = analysis_dataset_heir.at[index, ('HEADWATER_ANALYSIS_RESULTS', 'DEMAND_MET_HEADWATER_TOTAL')]

    for subwatershed in subwatersheds+summary_subtotals:
        supply_watershed_acct_value = analysis_dataset_heir.at[index, ('SUPPLY_WATERSHED_ACCT', subwatershed)]
        
        if index == 0:
            # Assign Watershed Supply Account for the first record
            if subwatershed in subwatersheds:
                # Get Supply Period Final for each subwatershed
                supply_period_final_value = hwa_outputs.loc['SUPPLY_PERIOD_FINAL', subwatershed]
                # Get Instream Flow Supply for each subwatershed
                instream_flow_supply_value = hwa_outputs.loc['INSTREAM_FLOW_SUPPLY', subwatershed]
        
                # Instream Flow Supply is not available to Riparian-only claims 
                if water_right_type_custom == 'Statement of Div and Use (Riparian)':
                    supply_watershed_acct_value = max(0, supply_period_final_value - instream_flow_supply_value)
            else: #subwatershed in summary_subtotals
                supply_watershed_acct_value = sum(analysis_dataset_heir.at[0, ('SUPPLY_WATERSHED_ACCT', sw)] for sw in subtotal_mapping[subwatershed])
            analysis_dataset_heir.at[index, ('SUPPLY_WATERSHED_ACCT', subwatershed)] = supply_watershed_acct_value
            continue

        # Calculate the Watershed Supply Account balance for subsequent records
        if subwatershed in subwatersheds:
            # The first record that is not Riparian-only also has access to the Instream Flow Supply
            if subwatershed_value == 'Sacramento Bend' and water_right_type_custom == 'Statement of Div and Use (Riparian or Pre-1914)':
                # Get the Instream Flow Suppl for each subwatershed
                instream_flow_supply_value = hwa_outputs.loc['INSTREAM_FLOW_SUPPLY', subwatershed]
                previous_supply_watershed_acct = analysis_dataset_heir.at[index-1, ('SUPPLY_WATERSHED_ACCT', subwatershed)]
                previous_demand_watershed = analysis_dataset_heir.at[index-1, ('DEMAND_WATERSHED', subwatershed)]
                supply_watershed_acct_value = max(0, previous_supply_watershed_acct - previous_demand_watershed) + instream_flow_supply_value
            
            # All subsequent records simply subtract the previous demand from the account
            else:
                previous_supply_watershed_acct = analysis_dataset_heir.at[index-1, ('SUPPLY_WATERSHED_ACCT', subwatershed)]
                previous_demand_watershed = analysis_dataset_heir.at[index-1, ('DEMAND_WATERSHED', subwatershed)]
                supply_watershed_acct_value = max(0, previous_supply_watershed_acct - previous_demand_watershed)
    
        # Watershed Supply Account subtotals
        else:
            supply_watershed_acct_value = sum(analysis_dataset_heir.at[index, ('SUPPLY_WATERSHED_ACCT', sw)] for sw in subtotal_mapping[subwatershed])
        analysis_dataset_heir.at[index, ('SUPPLY_WATERSHED_ACCT', subwatershed)] = supply_watershed_acct_value

    # Calculate Watershed Demand based on Watershed Supply Account values 
    # Some unique Watershed Supply Account values are needed for Watershed Demand calculations
    putah_supply = analysis_dataset_heir['SUPPLY_WATERSHED_ACCT', 'Putah'].values[index]
    sacramento_subtotal_supply = analysis_dataset_heir['SUPPLY_WATERSHED_ACCT', 'Sacramento Subtotal'].values[index]
    san_joaquin_subtotal_supply = analysis_dataset_heir['SUPPLY_WATERSHED_ACCT', 'San Joaquin Subtotal'].values[index]
    delta_tributary_subtotal_supply = analysis_dataset_heir['SUPPLY_WATERSHED_ACCT', 'Delta Tributary Subtotal'].values[index]
    sacramento_bend_supply = analysis_dataset_heir.at[index, ('SUPPLY_WATERSHED_ACCT', 'Sacramento Bend')]
    upper_sacramento_valley_supply = analysis_dataset_heir.at[index, ('SUPPLY_WATERSHED_ACCT', 'Upper Sacramento Valley')]

    for subwatershed in subwatersheds+summary_subtotals:
        supply_watershed_acct_value = analysis_dataset_heir.at[index, ('SUPPLY_WATERSHED_ACCT', subwatershed)]
    
    # Most comments below explaining details of the accounting are not repeated for other subwatersheds
              
        # Sacramento Bend and Upper Sacramento Valley
        if subwatershed in ['Sacramento Bend', 'Upper Sacramento Valley']:
            if legal_delta_value:
                if water_right_type_custom == 'Statement of Div and Use (Riparian)':
                    if watershed_value == 'Sacramento':
                        analysis_dataset_heir.at[index, ('DEMAND_WATERSHED', subwatershed)] = demand_met_headwater_value * \
                        safe_divide(supply_watershed_acct_value, (putah_supply + sacramento_subtotal_supply))
                    else:
                        analysis_dataset_heir.at[index, ('DEMAND_WATERSHED', subwatershed)] = 0
                else:
                    analysis_dataset_heir.at[index, ('DEMAND_WATERSHED', subwatershed)] = demand_met_headwater_value * \
                        safe_divide(supply_watershed_acct_value, (sacramento_subtotal_supply + san_joaquin_subtotal_supply + delta_tributary_subtotal_supply))
            elif headwater_value:
                # If the record is in Sacramento Bend, 100% of its Watershed Demand is assigned to Sacramento Bend
                if subwatershed_value == subwatershed:
                    analysis_dataset_heir.at[index, ('DEMAND_WATERSHED', subwatershed)] = demand_met_headwater_value
                else:
                    analysis_dataset_heir.at[index, ('DEMAND_WATERSHED', subwatershed)] = 0
            else:
                if subwatershed_value == 'Upper Sacramento Valley':
                    analysis_dataset_heir.at[index, ('DEMAND_WATERSHED', subwatershed)] = demand_met_headwater_value * \
                    safe_divide(supply_watershed_acct_value, (sacramento_bend_supply + upper_sacramento_valley_supply))
                elif watershed_value == 'Sacramento':
                    analysis_dataset_heir.at[index, ('DEMAND_WATERSHED', subwatershed)] = demand_met_headwater_value * \
                    safe_divide(supply_watershed_acct_value, sacramento_subtotal_supply)
                else:
                    analysis_dataset_heir.at[index, ('DEMAND_WATERSHED', subwatershed)] = 0

        # Stony, Cache, Upper Feather, Yuba, Bear, Upper American, and Sacramento Valley Floor
        elif subwatershed in ['Stony', 'Cache', 'Upper Feather', 'Yuba', 'Bear', 'Upper American', 'Sacramento Valley Floor']:
            if legal_delta_value:
                if water_right_type_custom == 'Statement of Div and Use (Riparian)':
                    if watershed_value == 'Sacramento':
                        analysis_dataset_heir.at[index, ('DEMAND_WATERSHED', subwatershed)] = demand_met_headwater_value * \
                        safe_divide(supply_watershed_acct_value, (putah_supply + sacramento_subtotal_supply))
                    else:
                        analysis_dataset_heir.at[index, ('DEMAND_WATERSHED', subwatershed)] = 0
                else:
                    analysis_dataset_heir.at[index, ('DEMAND_WATERSHED', subwatershed)] = demand_met_headwater_value * \
                    safe_divide(supply_watershed_acct_value, (sacramento_subtotal_supply + san_joaquin_subtotal_supply + delta_tributary_subtotal_supply))
            elif headwater_value and subwatershed_value == subwatershed:
                analysis_dataset_heir.at[index, ('DEMAND_WATERSHED', subwatershed)] = demand_met_headwater_value
            elif subwatershed_value == 'Upper Sacramento Valley':
                analysis_dataset_heir.at[index, ('DEMAND_WATERSHED', subwatershed)] = 0
            elif subwatershed_value == 'Sacramento Valley Floor':
                analysis_dataset_heir.at[index, ('DEMAND_WATERSHED', subwatershed)] = demand_met_headwater_value * \
                safe_divide(supply_watershed_acct_value, sacramento_subtotal_supply)
            else:
                analysis_dataset_heir.at[index, ('DEMAND_WATERSHED', subwatershed)] = 0

        # Upper San Joaquin, Fresno, Chowchilla, Merced, Tuolumne, and San Joaquin Valley Floor
        elif subwatershed in ['Upper San Joaquin', 'Fresno', 'Chowchilla', 'Merced', 'Tuolumne', 'San Joaquin Valley Floor']:
            if legal_delta_value:
                if water_right_type_custom == 'Statement of Div and Use (Riparian)':
                    if watershed_value == 'San Joaquin':
                        analysis_dataset_heir.at[index, ('DEMAND_WATERSHED', subwatershed)] = demand_met_headwater_value * \
                        safe_divide(supply_watershed_acct_value, (san_joaquin_subtotal_supply + delta_tributary_subtotal_supply - putah_supply))
                    else:
                        analysis_dataset_heir.at[index, ('DEMAND_WATERSHED', subwatershed)] = 0
                else:
                    analysis_dataset_heir.at[index, ('DEMAND_WATERSHED', subwatershed)] = demand_met_headwater_value * \
                    safe_divide(supply_watershed_acct_value, (sacramento_subtotal_supply + san_joaquin_subtotal_supply + delta_tributary_subtotal_supply))
            elif headwater_value and subwatershed_value == subwatershed:
                analysis_dataset_heir.at[index, ('DEMAND_WATERSHED', subwatershed)] = demand_met_headwater_value
            elif subwatershed_value == 'San Joaquin Valley Floor':
                analysis_dataset_heir.at[index, ('DEMAND_WATERSHED', subwatershed)] = demand_met_headwater_value * \
                safe_divide(supply_watershed_acct_value, (san_joaquin_subtotal_supply))
            else:
                analysis_dataset_heir.at[index, ('DEMAND_WATERSHED', subwatershed)] = 0

        # Putah
        elif subwatershed == 'Putah':
            if legal_delta_value:
                if water_right_type_custom == 'Statement of Div and Use (Riparian)':
                    if watershed_value == 'Sacramento':
                        analysis_dataset_heir.at[index, ('DEMAND_WATERSHED', subwatershed)] = demand_met_headwater_value * \
                        safe_divide(supply_watershed_acct_value, (putah_supply + sacramento_subtotal_supply))
                    else:
                        analysis_dataset_heir.at[index, ('DEMAND_WATERSHED', subwatershed)] = 0
                else:
                    analysis_dataset_heir.at[index, ('DEMAND_WATERSHED', subwatershed)] = demand_met_headwater_value * \
                    safe_divide(supply_watershed_acct_value, (sacramento_subtotal_supply + san_joaquin_subtotal_supply + delta_tributary_subtotal_supply))
            elif headwater_value and subwatershed_value == subwatershed:
                analysis_dataset_heir.at[index, ('DEMAND_WATERSHED', subwatershed)] = demand_met_headwater_value
            else:
                analysis_dataset_heir.at[index, ('DEMAND_WATERSHED', subwatershed)] = 0

        # Stanislaus, Calaveras, Mokelumne, and Cosumnes
        elif subwatershed in ['Stanislaus', 'Calaveras', 'Mokelumne', 'Cosumnes']:
            if legal_delta_value:
                if water_right_type_custom == 'Statement of Div and Use (Riparian)':
                    if watershed_value == 'San Joaquin':
                        analysis_dataset_heir.at[index, ('DEMAND_WATERSHED', subwatershed)] = demand_met_headwater_value * \
                        safe_divide(supply_watershed_acct_value, (san_joaquin_subtotal_supply + delta_tributary_subtotal_supply - putah_supply))
                    else:
                        analysis_dataset_heir.at[index, ('DEMAND_WATERSHED', subwatershed)] = 0
                else:
                    analysis_dataset_heir.at[index, ('DEMAND_WATERSHED', subwatershed)] = demand_met_headwater_value * \
                    safe_divide(supply_watershed_acct_value, (sacramento_subtotal_supply + san_joaquin_subtotal_supply + delta_tributary_subtotal_supply))
            elif headwater_value and subwatershed_value == subwatershed:
                analysis_dataset_heir.at[index, ('DEMAND_WATERSHED', subwatershed)] = demand_met_headwater_value
            else:
                analysis_dataset_heir.at[index, ('DEMAND_WATERSHED', subwatershed)] = 0
            
        # Summary Subtotals
        else:
            analysis_dataset_heir.at[index, ('DEMAND_WATERSHED', subwatershed)] = sum(analysis_dataset_heir.at[index, ('DEMAND_WATERSHED', sw)] for sw in subtotal_mapping[subwatershed])

In [ ]:
'''
For each record, calculate a Watershed Demand Met from each of the 20 subwatersheds
'''

for subwatershed in subwatersheds:
    supply_watershed_acct_col = analysis_dataset_heir[('SUPPLY_WATERSHED_ACCT', subwatershed)]
    demand_watershed_col = analysis_dataset_heir[('DEMAND_WATERSHED', subwatershed)]
    
    analysis_dataset_heir[('DEMAND_MET_WATERSHED', subwatershed)] = supply_watershed_acct_col - np.maximum(0, supply_watershed_acct_col - demand_watershed_col)

In [ ]:
# Sum the Total Watershed Demand Met, from all 20 subwatersheds, for each record
analysis_dataset_heir[('WATERSHED_ANALYSIS_RESULTS', 'DEMAND_MET_WATERSHED_TOTAL')] = analysis_dataset_heir[[
    ('DEMAND_MET_WATERSHED', subwatershed) for subwatershed in subwatersheds
]].sum(axis=1)

In [ ]:
# Watershed Unmet Demand cannot be calculated on a subwatershed basis for records in lower subwatershed or the Legal Delta
# because they are apportioned to individual subwatersheds based on the supply available from each
# Instead, only a Total Watershed Unmet Demand can be calculated

# Calculate Total Watershed Unmet Demand and Total Unmet Demand (headwater + watershed)
demand_unmet_watershed_total_col = ('WATERSHED_ANALYSIS_RESULTS', 'DEMAND_UNMET_WATERSHED_TOTAL')
demand_unmet_total_col = ('WATERSHED_ANALYSIS_RESULTS', 'DEMAND_UNMET_TOTAL')

analysis_dataset_heir[demand_unmet_watershed_total_col] = (
    analysis_dataset_heir[('HEADWATER_ANALYSIS_RESULTS', 'DEMAND_MET_HEADWATER_TOTAL')] - 
    analysis_dataset_heir[('WATERSHED_ANALYSIS_RESULTS', 'DEMAND_MET_WATERSHED_TOTAL')]
)

analysis_dataset_heir[demand_unmet_total_col] = (
    analysis_dataset_heir[('HEADWATER_ANALYSIS_RESULTS', 'DEMAND_UNMET_HEADWATER_TOTAL')] +
    analysis_dataset_heir[demand_unmet_watershed_total_col]
)

In [ ]:
# Determine if Water Unavailable in Watershed (T/F) for each record
def calculate_water_unavailability(row, hwa_outputs):
    
    if not row[('HEADWATER_ANALYSIS_RESULTS', 'WATER_UNAVAILABLE_HEADWATER')]:
        disconnected = hwa_outputs.loc['DISCONNECTED', row[('', 'SUBWATERSHED')]]
        
        if not row[('', 'LEGAL_DELTA')]:
            # If subwatershed is Disconnected, value is false
            if disconnected:
                return False
            else:
                # For records in a headwater subwatershed outside the Legal Delta, unavailability is based only on local supply
                if row[('', 'HEADWATER')]:
                    # If zero Watershed Supply Account is available to a right, value is true
                    return row[('SUPPLY_WATERSHED_ACCT', row[('', 'SUBWATERSHED')])] == 0
                
                # For records in lower subwatersheds and/or the Legal Delta, unavailability is more complicated
                else:
                    # Upper Sacramento Valley has access to Sacramento Bend and Upper Sacramento Valley supply
                    if row[('', 'SUBWATERSHED')] == 'Upper Sacramento Valley':
                        return row[('SUPPLY_WATERSHED_ACCT', 'Sacramento Bend')] == 0 and row[('SUPPLY_WATERSHED_ACCT', 'Upper Sacramento Valley')] == 0
                    # Sacramento Valley Floor has access to all Sacramento tributaries
                    elif row[('', 'SUBWATERSHED')] == 'Sacramento Valley Floor':
                        return row[('SUPPLY_WATERSHED_ACCT', 'Sacramento Subtotal')] == 0
                    # San Joaquin Valley Floor has access to all San Joaquin tributaries
                    elif row[('', 'SUBWATERSHED')] == "San Joaquin Valley Floor":
                        return row[('SUPPLY_WATERSHED_ACCT', 'San Joaquin Subtotal')] == 0
        
        # Legal Delta has access to all upstream subwatersheds
        else:
            # Riparian-only claims in Legal Delta only have access to subwatersheds in their watershed
            if row[('', 'WATER_RIGHT_TYPE_CUSTOM')] == 'Statement of Div and Use (Riparian)':
                if row[('', 'WATERSHED')] == 'Sacramento':
                    # Putah isn't included in Sacramento subtotal since it is directly tributary to Legal Delta
                    return (row[('SUPPLY_WATERSHED_ACCT', 'Sacramento Subtotal')] +
                    row[('SUPPLY_WATERSHED_ACCT', 'Putah')]) == 0
                else: # San Joaquin watershed
                    # Delta Tributaries aren't included in San Joaquin subtotal
                    return (row[('SUPPLY_WATERSHED_ACCT', 'San Joaquin Subtotal')] +
                    row[('SUPPLY_WATERSHED_ACCT', 'Delta Tributary Subtotal')] -
                    row[('SUPPLY_WATERSHED_ACCT', 'Putah')]) == 0
            
            # Other Riparian claims in Legal Delta have access to all 20 subwatersheds
            else:
                return (row[('SUPPLY_WATERSHED_ACCT', 'San Joaquin Subtotal')] +
                row[('SUPPLY_WATERSHED_ACCT', 'Delta Tributary Subtotal')] +
                row[('SUPPLY_WATERSHED_ACCT', 'Sacramento Subtotal')]) == 0
    #This field indicates if water is unavailable only at the watershed scale
    # So if water is unavailable at the headwater scale, it is false
    else: 
        return False

# Apply the function to each row
analysis_dataset_heir[('WATERSHED_ANALYSIS_RESULTS', 'WATER_UNAVAILABLE_WATERSHED')] = analysis_dataset_heir.apply(
    lambda row: calculate_water_unavailability(row, hwa_outputs), 
    axis=1
)

In [ ]:
# Determine if Water Unavailable at Either the Headwater or Watershed scale
analysis_dataset_heir[('WATERSHED_ANALYSIS_RESULTS', 'WATER_UNAVAILABLE_EITHER')] = (
    analysis_dataset_heir['HEADWATER_ANALYSIS_RESULTS', 'WATER_UNAVAILABLE_HEADWATER'] | 
    analysis_dataset_heir['WATERSHED_ANALYSIS_RESULTS', 'WATER_UNAVAILABLE_WATERSHED']
)

In [ ]:
# Export intermediate results for use in 4.Curtailments script
analysis_dataset_heir.to_csv('./intermediate-outputs/Analysis_Results.csv', index=False)